In [5]:

!pip install -q -U google-generativeai
import google.generativeai as genai
import os
from datetime import datetime
import json


from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        raise ValueError("API Key não encontrada nos Secrets do Colab.")
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key configurada com sucesso!\n" + "="*30 + "\n")
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    print("Certifique-se de que você adicionou 'GOOGLE_API_KEY' nos Secrets do Colab e que ela é válida.\n" + "="*30 + "\n")
    GOOGLE_API_KEY = None


if GOOGLE_API_KEY:
    generation_config = {
        "candidate_count": 1,
        "temperature": 0.7,
        "top_p": 0.9,
        "top_k": 40,
    }

    safety_settings = {
        "HARASSMENT": "BLOCK_MEDIUM_AND_ABOVE",
        "HATE": "BLOCK_MEDIUM_AND_ABOVE",
        "SEXUAL": "BLOCK_MEDIUM_AND_ABOVE",
        "DANGEROUS": "BLOCK_MEDIUM_AND_ABOVE",
    }

    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        generation_config=generation_config,
        safety_settings=safety_settings
    )

    chat_session = model.start_chat(history=[])
    print("Modelo Gemini carregado e sessão de chat iniciada.\n" + "="*30 + "\n")
else:
    chat_session = None
    print("Modelo Gemini não pôde ser carregado devido à falha na configuração da API Key.\n" + "="*30 + "\n")


nome_preferido_usuario = None

lembrancas_db = {
    "viagem à praia": "{nome_usuario}, lembra daquela nossa viagem à praia no ano passado? O sol estava tão gostoso e a água do mar tão relaxante. Qual foi a sua parte favorita daquele dia?",
    "aniversário": "Ah, seu aniversário, {nome_usuario}! Uma data tão especial. Qual foi o seu bolo preferido que já tivemos?",
    "jardim florido": "Seu jardim era tão lindo, {nome_usuario}. Quais eram as flores que você mais gostava de cuidar?",
    "receita favorita": "{nome_usuario}, qual é aquela sua receita que todo mundo adora? Você ainda se lembra de todos os ingredientes?"
}

pessoas_queridas_db = {
    "ana": "{nome_usuario}, Ana é sua filha. Quais são as lembranças mais felizes que você tem com ela?",
    "carlos": "Carlos é seu neto, {nome_usuario}. O que vocês mais gostam de fazer juntos?",
    "elena": "Elena é uma amiga muito querida, {nome_usuario}. Como vocês se conheceram?",
    "júlio": "Júlio é seu irmão, {nome_usuario}. Compartilhem alguma história engraçada que viveram juntos."
}

frases_motivacionais_db = {
    "alegria": [
        "A felicidade reside não nas posses, e sim na alma.",
        "Cultive alegria em cada pequeno instante.",
        "Um sorriso pode iluminar o dia de alguém.",
        "A vida é um presente, desembrulhe cada momento com gratidão.",
        "Sinta a brisa suave da felicidade em seu coração."
    ],
    "tristeza": [
        "Mesmo a noite mais escura precede um amanhecer radiante.",
        "A tristeza é uma ponte para a compreensão mais profunda.",
        "Permita-se sentir, mas não deixe a tristeza te consumir.",
        "Você é mais forte do que imagina, essa fase vai passar.",
        "Conte com o tempo, ele cura muitas feridas."
    ],
    "frustracao": [
        "A frustração é apenas um sinal de que você se importa.",
        "Transforme seus obstáculos em degraus para o sucesso.",
        "Cada tentativa falha te ensina um novo caminho.",
        "A persistência conquista o que a inteligência não alcança.",
        "Respire fundo e lembre-se do seu valor."
    ],
    "calma": [
        "A quietude da mente traz clareza ao coração.",
        "Encontre seu centro e deixe a paz te guiar.",
        "A serenidade é um refúgio em meio ao caos.",
        "Relaxe, o universo está no seu ritmo.",
        "Acalme sua mente e ouça a sabedoria interior."
    ]
}

print("Bancos de dados simulados e variáveis globais carregados.\n" + "="*30 + "\n")


def enviar_mensagem_chat(prompt_usuario_formatado, chat_session_obj):
    """Envia uma mensagem para a sessão de chat e retorna a resposta do modelo."""
    if not chat_session_obj:
        return "Desculpe, não consigo processar sua mensagem pois a sessão com a IA não foi iniciada."
    try:
        response = chat_session_obj.send_message(prompt_usuario_formatado)
        return response.text
    except Exception as e:
        return f"Desculpe, tive um problema ao processar sua solicitação com a IA: {e}"

def formatar_nome_usuario(texto):
    """Formata o texto substituindo {nome_usuario} pelo nome preferido."""
    global nome_preferido_usuario
    return texto.format(nome_usuario=nome_preferido_usuario if nome_preferido_usuario else "você")

def conversar_amigavelmente(chat_sess, nome_usr):
    """Engaja em uma conversa amigável e estimulante."""
    prompt_para_gemini = f"""
    Contexto: {nome_usr} gostaria de conversar.
    Você é um amigo leal de {nome_usr}, oferecendo conversas leves e estimulantes.
    Inicie a conversa de forma gentil e faça uma pergunta aberta para encorajá-lo(a) a falar.
    Você pode perguntar sobre o dia dele(a), um hobby, algo que o(a) faz feliz, ou propor uma brincadeira simples para exercitar o raciocínio de forma suave.
    Mantenha um tom acolhedor, paciente e adaptado ao ritmo de {nome_usr}.
    Exemplo de início: "Olá, {nome_usr}! Como está se sentindo hoje? Há algo de bom que aconteceu?"
    Use o histórico da conversa para manter o contexto e evitar repetições.
    """
    return enviar_mensagem_chat(prompt_para_gemini, chat_sess)

def relembrar_memorias_usuario(termo_chave_usuario, chat_sess, nome_usr):
    """Busca uma lembrança e a desenvolve com o usuário."""
    termo_chave_usuario_lower = termo_chave_usuario.lower()
    lembranca_base_encontrada = None
    for chave_db, memoria_template in lembrancas_db.items():
        if termo_chave_usuario_lower in chave_db:
            lembranca_base_encontrada = formatar_nome_usuario(memoria_template)
            break

    if lembranca_base_encontrada:
        prompt_para_gemini = f"""
        Contexto: {nome_usr} pediu para relembrar algo sobre '{termo_chave_usuario}'.
        A lembrança inicial é: '{lembranca_base_encontrada}'.
        Desenvolva essa lembrança com {nome_usr}. Mencione a lembrança brevemente e faça perguntas sutis e gentis para estimular {nome_usr} a recordar mais detalhes.
        Seja paciente e ofereça dicas graduais se ele(a) tiver dificuldades, sempre priorizando o estímulo mental e o bem-estar emocional.
        Exemplo de pergunta: "Que mais você se lembra sobre aquele dia, {nome_usr}?" ou "Como você se sentiu naquele momento?"
        Use o histórico da conversa para evitar repetições e manter o fluxo da lembrança.
        """
        return enviar_mensagem_chat(prompt_para_gemini, chat_sess)
    else:
        return f"Não tenho uma lembrança específica sobre '{termo_chave_usuario}' no momento, {nome_usr}. Gostaria de falar sobre outra coisa ou tentar lembrar juntos?"

def lembrar_pessoa_querida_usuario(nome_pessoa, chat_sess, nome_usr):
    """Ajuda o usuário a relembrar informações sobre pessoas queridas."""
    nome_pessoa_lower = nome_pessoa.lower()
    info_pessoa_encontrada = None
    for chave_db, info_template in pessoas_queridas_db.items():
        if nome_pessoa_lower in chave_db:
            info_pessoa_encontrada = formatar_nome_usuario(info_template)
            break

    if info_pessoa_encontrada:
        prompt_para_gemini = f"""
        Contexto: {nome_usr} perguntou sobre '{nome_pessoa}'.
        A informação inicial é: '{info_pessoa_encontrada}'.
        Incentive {nome_usr} a lembrar mais sobre '{nome_pessoa}'. Faça perguntas sutis e gentis sobre essa pessoa, como suas características, momentos compartilhados ou sentimentos associados a ela.
        Demonstre paciência e compreensão, respeitando o ritmo de {nome_usr}. Ofereça dicas graduais se ele(a) tiver dificuldades, sempre buscando o estímulo mental e a conexão emocional.
        Exemplo de pergunta: "O que mais você lembra sobre a {nome_pessoa}?" ou "O que essa pessoa significa para você, {nome_usr}?"
        Use o histórico da conversa para evitar repetições e aprofundar a lembrança.
        """
        return enviar_mensagem_chat(prompt_para_gemini, chat_sess)
    else:
        return f"Não tenho informações detalhadas sobre '{nome_pessoa}' agora, {nome_usr}. Quem é essa pessoa especial para você?"

def obter_frases_motivacionais(categoria):
    """Retorna frases motivacionais com base na categoria."""
    if categoria.lower() in frases_motivacionais_db:
        return "\n".join(frases_motivacionais_db[categoria.lower()])
    else:
        return "Desculpe, não tenho frases motivacionais para essa categoria no momento."

print("Funções dos módulos atualizadas e prontas.\n" + "="*30 + "\n")


def chatbot_guardiao_das_memorias_v4():
    global nome_preferido_usuario
    global chat_session

    if not chat_session:
        print("Guardião: Desculpe, não consigo iniciar. Parece que há um problema com a conexão à IA. Verifique a API Key.")
        return

    print("\n--- Guardião das Memórias ---")
    print("Seja bem-vindo(a)!")

    while nome_preferido_usuario is None:
        nome_temp = input("Guardião: Como você gostaria de ser chamado(a)? ").strip()
        if nome_temp:
            nome_preferido_usuario = nome_temp
            print(f"\nGuardião: Olá, {nome_preferido_usuario}! É um prazer conhecê-lo(a).\n"
                  f"    Estou aqui para conversar, ajudar a lembrar e ser um amigo. 😊")
            chat_session.history.append({'role': 'user', 'parts': [f"Meu nome é {nome_preferido_usuario}."] })
            chat_session.history.append({'role': 'model', 'parts': [f"Entendido, {nome_preferido_usuario}!"] })
        else:
            print("Guardião: Por favor, me diga um nome para que eu possa te chamar assim.")
        print("-" * 30 + "\n")

    print(f"Guardião: {nome_preferido_usuario}, como posso te ajudar hoje?")
    print("Você pode escolher uma das opções abaixo digitando o número ou a palavra:")
    print("1. Conversar")
    print("2. Relembrar Memórias")
    print("3. Pessoas Queridas")
    print("4. Frases Motivacionais")
    print("\nDigite 'sair' para terminar.")
    print("=" * 30 + "\n")

    while True:
        try:
            entrada_usuario = input(f"{nome_preferido_usuario}: ").strip()
            if not entrada_usuario:
                continue

            print("-" * 30)

            entrada_lower = entrada_usuario.lower()

            if entrada_lower == "sair":
                resposta_despedida = enviar_mensagem_chat(f"O usuário {nome_preferido_usuario} está se despedindo. Dê uma mensagem de despedida calorosa e gentil.", chat_session)
                print(f"\nGuardião:\n    {resposta_despedida}\n")
                break
            elif entrada_lower == "1" or "conversar" in entrada_lower:
                resposta = conversar_amigavelmente(chat_session, nome_preferido_usuario)
                print(f"\nGuardião:\n    {resposta}\n")
            elif entrada_lower == "2" or "relembrar memórias" in entrada_lower or "memória" in entrada_lower:
                termo = input("Guardião: Sobre o que você gostaria de relembrar? ").strip()
                resposta = relembrar_memorias_usuario(termo, chat_session, nome_preferido_usuario)
                print(f"\nGuardião:\n    {resposta}\n")
            elif entrada_lower == "3" or "pessoas queridas" in entrada_lower or "pessoa querida" in entrada_lower:
                nome = input("Guardião: Quem você gostaria de relembrar? ").strip()
                resposta = lembrar_pessoa_querida_usuario(nome, chat_session, nome_preferido_usuario)
                print(f"\nGuardião:\n    {resposta}\n")
            elif entrada_lower == "4" or "frases motivacionais" in entrada_lower or "motivação" in entrada_lower or "ânimo" in entrada_lower:
                print("Guardião: Para qual estado de espírito você gostaria de ler frases?")
                print("Opções: Alegria, Tristeza, Frustracao, Calma")
                categoria_motivacional = input(f"{nome_preferido_usuario}: ").strip()
                frases = obter_frases_motivacionais(categoria_motivacional)
                print(f"\nGuardião:\n    {frases}\n")
            else:
                prompt_geral = f"O usuário {nome_preferido_usuario} disse: '{entrada_usuario}'. Responda de forma amigável e compreensiva, como um assistente atencioso. Se não entender bem, peça para ele explicar de outra forma ou sugira usar os comandos do menu (conversar, relembrar memórias, pessoas queridas, frases motivacionais)."
                resposta = enviar_mensagem_chat(prompt_geral, chat_session)
                print(f"\nGuardião:\n    {resposta}\n")

        except KeyboardInterrupt:
            print(f"\n\nGuardião: Entendido, {nome_preferido_usuario}. Encerrando nossa conversa por agora. Se cuide!\n")
            break
        except Exception as e:
            print(f"\nGuardião: Ops, encontrei um pequeno problema técnico: {e}. Vamos tentar de novo?\n")
        print("=" * 30 + "\n")


if __name__ == "__main__" and chat_session:
    chatbot_guardiao_das_memorias_v4()
else:
    if not GOOGLE_API_KEY:
        print("Não é possível iniciar o chatbot: A API Key do Google não foi configurada corretamente.")
    elif not chat_session:
        print("Não é possível iniciar o chatbot: A sessão de chat com o Gemini não foi iniciada. Verifique as células anteriores.")

API Key configurada com sucesso!

Modelo Gemini carregado e sessão de chat iniciada.

Bancos de dados simulados e variáveis globais carregados.

Funções dos módulos atualizadas e prontas.


--- Guardião das Memórias ---
Seja bem-vindo(a)!
Guardião: Como você gostaria de ser chamado(a)? Ryan

Guardião: Olá, Ryan! É um prazer conhecê-lo(a).
    Estou aqui para conversar, ajudar a lembrar e ser um amigo. 😊
------------------------------

Guardião: Ryan, como posso te ajudar hoje?
Você pode escolher uma das opções abaixo digitando o número ou a palavra:
1. Conversar
2. Relembrar Memórias
3. Pessoas Queridas
4. Frases Motivacionais

Digite 'sair' para terminar.

Ryan: 2
------------------------------
Guardião: Sobre o que você gostaria de relembrar? viagem à praia

Guardião:
    Ryan, tudo bem por aí? Estava aqui pensando na nossa viagem à praia do ano passado. Lembra daquele sol gostoso e da água do mar relaxante?

Que mais vem à sua mente sobre aquele dia? Alguma imagem, cheiro ou sensaçã